In [1]:
from expression_tree import ExpressionTree
import numpy as np
import pandas as pd
import pyomo.environ as pyo

In [2]:
z = ExpressionTree({
    1: "sum",
    2: "mult",
    3: "sum",
    4: "cst",
    5: "exp",
    6: "cst",
    7: "mult",
    11: "x1",
    14: "cst",
    15: "x2",
})

In [3]:
print(z)

c_14*x2 + c_4*exp(x1) + c_6


In [4]:
# Generate data for parameter estimation
np.random.seed(42)
data = pd.DataFrame(
    np.random.uniform(low=1, high=3, size=(10, 2)), columns=["x1", "x2"]
)
data["y"] = 2 * np.exp(data["x1"]) - 3.5 * data["x2"] + 2.478
data

,x1,x2,y
0,1.749080,2.901429,3.821624
1,2.463988,2.197317,18.290555
2,1.312037,1.311989,5.313502
3,1.116167,2.732352,-0.978973
4,2.202230,2.416145,12.111816
5,1.041169,2.939820,-2.146316
6,2.664885,1.424678,26.224229
7,1.363650,1.366809,5.515049
8,1.608484,2.049513,5.295175
9,1.863890,1.582458,9.836944


In [5]:
m = z.get_parameter_estimation_model(data)

List of parameters in the expression: ['c_4', 'c_6', 'c_14']


In [6]:
# Optional: Add bounds on the parameters.
# Bounds are needed if the expression is nonlinear and we are
# interested in solving the problem to global optimality
m.c_4.bounds = (-10, 10)
m.c_14.bounds = (-10, 10)
m.c_6.bounds = (-10, 10)

In [7]:
# Solve the optimization problem
solver = pyo.SolverFactory("scip")
solver.solve(m, tee=True)

SCIP version 9.0.0 [precision: 8 byte] [memory: block] [mode: optimized] [LP solver: Soplex 7.0.0] [GitHash: 7205bedd94]
Copyright (c) 2002-2024 Zuse Institute Berlin (ZIB)

External libraries: 
  Soplex 7.0.0         Linear Programming Solver developed at Zuse Institute Berlin (soplex.zib.de) [GitHash: 6657fb3b]
  CppAD 20180000.0     Algorithmic Differentiation of C++ algorithms developed by B. Bell (github.com/coin-or/CppAD)
  ZLIB 1.2.13          General purpose compression library by J. Gailly and M. Adler (zlib.net)
  MPIR 3.0.0           Multiple Precision Integers and Rationals Library developed by W. Hart (mpir.org)
  ZIMPL 3.5.3          Zuse Institute Mathematical Programming Language developed by T. Koch (zimpl.zib.de)
  AMPL/MP 690e9e7      AMPL .nl file reader library (github.com/ampl/mp)
  PaPILO 2.2.0         parallel presolve for integer and linear optimization (github.com/scipopt/papilo) (built with TBB) [GitHash: df30ae49]
  bliss 0.77           Computing Graph Autom

{'Problem': [{'Lower bound': -8.99990760845011e-07, 'Upper bound': -8.99990760845011e-07, 'Number of objectives': 1, 'Number of constraints': 0, 'Number of variables': 53, 'Sense': 'unknown'}], 'Solver': [{'Status': 'ok', 'Message': 'optimal solution found', 'Termination condition': 'optimal', 'Id': 0, 'Error rc': 0, 'Time': 0.0, 'Gap': 0.0, 'Primal bound': -8.99990760845011e-07, 'Dual bound': -8.99990760845011e-07}], 'Solution': [OrderedDict({'number of solutions': 0, 'number of solutions displayed': 0})]}

In [8]:
print("Optimal value of c_4: ", m.c_4.value)
print("Optimal value of c_6: ", m.c_6.value)
print("Optimal value of c_14: ", m.c_14.value)

Optimal value of c_4:  2.0000000000000244
Optimal value of c_6:  2.477999999999223
Optimal value of c_14:  -3.499999999999717
